## bleu score

In [390]:
%matplotlib inline 

In [391]:
run -i bleu.py

ERROR:root:File `'bleu.py'` not found.


In [392]:
  
import re
import random
import collections
import torch
import numpy as np
import nltk

#%%
with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/test_source.txt", "r", encoding="utf8") as f:
    
    source = []
    
    for i, line in enumerate(f):
        
        line = line.split('<JOIN>')
        source.append(line[0])
        
  
    
with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/test_target.txt", "r", encoding="utf8") as f:
    
    targets = []
    
    for i, line in enumerate(f):
        
        line = line.split('<JOIN>')
        targets.append(line[0])
        
        
    
with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/test_pred2.txt", "r", encoding="utf8") as f:
    
    predictions = []
    s2=[]
    
    for i, line in enumerate(f):
        s1=(line.split('  '))
        #print(s1)
        for k, word in enumerate(s1):
            s2.append(" ".join(["".join(w.split(" ")) for w in word.split("  ")]))
        predictions.append(" ".join(s2))
        s2=[]
 



In [393]:
#for the QRNN

with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/predicted_CNN.txt", "r", encoding="utf8") as f:
    
    pred_cnn = []
    
    for i, line in enumerate(f):
        
        line = line.split('<JOIN>')
        pred_cnn.append(line[0])
        
with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/target_CNN.txt", "r", encoding="utf8") as f:
    
    target_cnn = []
    
    for i, line in enumerate(f):
        
        line = line.split('<JOIN>')
        target_cnn.append(line[0])
        
        
with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/source_CNN.txt", "r", encoding="utf8") as f:
    
    source_cnn = []
    
    for i, line in enumerate(f):
        
        line = line.split('<JOIN>')
        source_cnn.append(line[0])

In [394]:
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" ", s) #separates punctuation from the word
    #s = re.sub(r"([.!?])", r" \1", s) #separates punctuation from the word
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s) #strips anything that isn't a character of punctuation
    
    return s

In [395]:
#get rid of /n 
source_u=[]
for i, word in enumerate(source): 
    source_u.append(normalizeString(word.strip()))
    
target_u=[]
for i, t in enumerate(targets):
    target_u.append(normalizeString(t.strip()))
    
pred_u=[]
for i, p in enumerate(predictions):
    pred_u.append(normalizeString(p.strip()))  


In [410]:
#for cnn 
pred_u1=[]
for i, p in enumerate(pred_cnn):
    pred_u1.append(normalizeString(p.strip()))  

target_u1=[]
for i, p in enumerate(target_cnn):
    target_u1.append(normalizeString(p.strip()))  

source_u1=[]
for i, p in enumerate(source_cnn):
    source_u1.append(normalizeString(p.strip()))  


In [413]:
print(len(source_u1))
print(len(target_u1))
print(len(pred_u1))


2624
2624
2624


In [412]:
source_u1=source_u1[0:2624]

In [403]:
print(target_u[800])
print('')
print(pred_u[800])


we type we file we go to dinner it s fine 

we typing them we re eating that is okay that s okay 


In [400]:
#bleu score
b=moses_multi_bleu(pred_u1, target_u1, lowercase=False)
print(b)

2.49


In [399]:
with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/compounds.txt", "r", encoding="utf8") as comp:
    compound_words = []
    
    for i,line in enumerate(comp):        
        compound_words.append(line.strip())

In [414]:
#experiment-1: find the sentences where the compound words occurs and  
#find the  bleu score for those
pred_comp=[]
target_comp=[]

for i in range(len(pred_u1)):
    for w in source_u1[i].split():
        if w in compound_words:
            #print(w)
            #print(i)
            pred_comp.append(pred_u1[i])
            target_comp.append(target_u1[i])
            break
                  


In [382]:
print(source_u[949]) 
print('')
print(target_u[949 ]) 
print('')
print(pred_u[949])

weil wir keine gene herumschieben wollen 

because we don t want to move genes around 

because we want to shift around the way we want to do that 


In [385]:
#find when qrnn output is given . 
ind=1
for w in target_u1:
    ind=ind+1
    if w=='because we don t want to move genes around ':
        print(ind)

932


In [389]:
pred_u1[931]

'it s not the too e n t i n t i tis ic t ic t ic t ic '

In [277]:
blue_comp=moses_multi_bleu(pred_comp, target_comp, lowercase=False)

print(blue_comp)

228
2.54


In [415]:
#experiment-2: find the sentences where the compound words occurs and 
#count the number of words in each sentence.
target_length=[]
source_length=[]
pred_length=[]

t=[]
s=[]
p=[]

for i in range(len(pred_u1)):
    for w in source_u1[i].split():
        if w in compound_words:
            target_length.append(len(target_u1[i].split()))
            source_length.append(len(source_u1[i].split())) 
            pred_length.append(len(pred_u1[i].split())) 

            t.append(target_u1[i])
            s.append(source_u1[i])
            p.append(pred_u1[i])

            break
         

print(sum(target_length) / float(len(target_length)))
print(sum(source_length) / float(len(source_length)))
print(sum(pred_length) / float(len(pred_length)))


18.420654911838792
19.2191435768262
16.17632241813602


In [416]:
import numpy as np
from scipy import stats 

t_l = np.array(target_length)
s_l = np.array(source_length)
p_l = np.array(pred_length)

print(stats.sem(t_l))
print(stats.sem(s_l))
print(stats.sem(p_l))




0.552193702963
0.500971847087
0.61058925154


In [208]:
#not using this anymore

target_noncomp_length=[]
source_noncomp_length=[]
pred_noncomp_length=[]

#find the non compound sentences 
for i in range(len(target_u)):
    for sentence in target_u:
        if sentence in t:
            break
        else:
            target_noncomp_length.append(len(target_u[i].split()))
        
for i in range(len(source_u)):
    for sentence in source_u:
        if sentence in s:
            break
        else:
            source_noncomp_length.append(len(source_u[i].split()))
            
for i in range(len(pred_u)):
    for sentence in pred_u:
        if sentence in s:
            break
        else:
            pred_noncomp_length.append(len(pred_u[i].split()))
 
 

In [ ]:
print(sum(target_noncomp_length) / float(len(target_noncomp_length)))
print(sum(source_noncomp_length) / float(len(source_noncomp_length)))
print(sum(pred_noncomp_length) / float(len(pred_noncomp_length)))

In [417]:
#overall length
overall_source_length=[]
overall_target_length=[]
overall_pred_length=[]


for i in range(len(source_u1)):
    overall_source_length.append(len(source_u1[i].split()))
    overall_target_length.append(len(target_u1[i].split()))
    overall_pred_length.append(len(pred_u1[i].split()))

            
print(sum(overall_source_length) / float(len(overall_source_length)))
print(sum(overall_target_length) / float(len(overall_target_length)))
print(sum(overall_pred_length) / float(len(overall_pred_length)))

t_l = np.array(overall_target_length)
s_l = np.array(overall_source_length)
p_l = np.array(overall_pred_length)

print(stats.sem(t_l))
print(stats.sem(s_l))
print(stats.sem(p_l))



14.036966463414634
15.754954268292684
14.690167682926829
0.201143566814
0.178747859663
0.217527359537


In [418]:
#compare long vs short sentences
#use 10 as cutoff

long_sent_pred=[]
long_sent_target=[]
short_sent_pred=[]
short_sent_target=[]

for i in range(len(pred_u1)):
    if len(source_u1[i])>100:
        long_sent_pred.append(pred_u1[i])
        long_sent_target.append(target_u1[i])
    else:
        short_sent_pred.append(pred_u1[i])
        short_sent_target.append(target_u1[i])



In [419]:
blue_long=moses_multi_bleu(long_sent_pred, long_sent_target, lowercase=False)
print(blue_long) 

blue_short=moses_multi_bleu(short_sent_pred, short_sent_target, lowercase=False)
print(blue_short)

1.93
2.89
